In [4]:
!pip install streamlit

In [5]:
import streamlit as st
import pandas as pd
import numpy as np
from datetime import datetime
import uuid
import json
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Mock AI vision API (simulating satellite imagery analysis)
def analyze_rooftop_image(image_file, location):
    try:
        # Simulated AI vision output (replace with real API like OpenRouter in production)
        return {
            "roof_area_sqft": np.random.uniform(500, 2000),  # Random roof area for demo
            "shading_percentage": np.random.uniform(0, 30),   # Random shading
            "roof_angle_degrees": np.random.uniform(15, 45),  # Random roof angle
            "solar_potential_kwh_per_year": np.random.uniform(5000, 15000),  # Random solar potential
            "confidence_score": np.random.uniform(0.8, 0.95)  # Random confidence
        }
    except Exception as e:
        logger.error(f"Error in image analysis: {str(e)}")
        raise Exception("Failed to analyze rooftop image")

# Solar panel specifications
SOLAR_PANEL_SPEC = {
    "efficiency": 0.20,  # 20% efficiency
    "wattage_per_panel": 400,  # 400W per panel
    "area_per_panel_sqft": 20,  # 20 sqft per panel
    "cost_per_watt": 0.7,  # $0.70 per watt
    "installation_cost_per_watt": 0.5,  # $0.50 per watt
    "maintenance_cost_per_year": 100,  # Annual maintenance
    "panel_lifespan_years": 25  # 25 years
}

# ROI calculation
def calculate_roi(roof_area, solar_potential, electricity_rate, incentives):
    try:
        max_panels = int(roof_area / SOLAR_PANEL_SPEC["area_per_panel_sqft"])
        system_size_kw = max_panels * SOLAR_PANEL_SPEC["wattage_per_panel"] / 1000
        total_cost = system_size_kw * 1000 * (SOLAR_PANEL_SPEC["cost_per_watt"] + SOLAR_PANEL_SPEC["installation_cost_per_watt"])
        total_cost -= incentives  # Apply incentives
        annual_savings = solar_potential * electricity_rate
        annual_net_savings = annual_savings - SOLAR_PANEL_SPEC["maintenance_cost_per_year"]
        payback_period = total_cost / annual_net_savings if annual_net_savings > 0 else float('inf')
        lifetime_savings = annual_net_savings * SOLAR_PANEL_SPEC["panel_lifespan_years"] - total_cost

        return {
            "system_size_kw": round(system_size_kw, 2),
            "total_cost_usd": round(total_cost, 2),
            "annual_savings_usd": round(annual_savings, 2),
            "payback_period_years": round(payback_period, 2),
            "lifetime_savings_usd": round(lifetime_savings, 2)
        }
    except Exception as e:
        logger.error(f"Error in ROI calculation: {str(e)}")
        raise Exception("Failed to calculate ROI")

# Streamlit app
def main():
    st.set_page_config(page_title="Solar Potential Analysis", layout="wide")
    st.title("AI-Powered Rooftop Solar Analysis")
    st.markdown("Upload a satellite image of your rooftop to assess solar installation potential.")

    # Input form
    with st.form("solar_form"):
        location = st.text_input("Location (City, State)", value="San Francisco, CA")
        image_file = st.file_uploader("Upload Satellite Image", type=["png", "jpg", "jpeg"])
        electricity_rate = st.number_input("Electricity Rate ($/kWh)", min_value=0.01, max_value=1.0, value=0.15, step=0.01)
        incentives = st.number_input("Available Incentives ($)", min_value=0.0, value=5000.0, step=100.0)
        submit = st.form_submit_button("Analyze")

    if submit and image_file:
        try:
            with st.spinner("Analyzing rooftop..."):
                # Analyze image
                analysis_result = analyze_rooftop_image(image_file, location)
                
                # Display results
                st.subheader("Rooftop Analysis Results")
                col1, col2 = st.columns(2)
                with col1:
                    st.metric("Roof Area (sqft)", f"{analysis_result['roof_area_sqft']:.2f}")
                    st.metric("Shading (%)", f"{analysis_result['shading_percentage']:.2f}")
                with col2:
                    st.metric("Roof Angle (degrees)", f"{analysis_result['roof_angle_degrees']:.2f}")
                    st.metric("Annual Solar Potential (kWh)", f"{analysis_result['solar_potential_kwh_per_year']:.2f}")
                st.metric("Confidence Score", f"{analysis_result['confidence_score']:.2%}")

                # ROI Analysis
                roi_result = calculate_roi(
                    analysis_result['roof_area_sqft'],
                    analysis_result['solar_potential_kwh_per_year'],
                    electricity_rate,
                    incentives
                )
                
                st.subheader("Financial Analysis")
                col3, col4 = st.columns(2)
                with col3:
                    st.metric("System Size (kW)", roi_result["system_size_kw"])
                    st.metric("Total Cost ($)", f"{roi_result['total_cost_usd']:,.2f}")
                with col4:
                    st.metric("Annual Savings ($)", f"{roi_result['annual_savings_usd']:,.2f}")
                    st.metric("Payback Period (years)", roi_result["payback_period_years"])
                st.metric("Lifetime Savings ($)", f"{roi_result['lifetime_savings_usd']:,.2f}")

                # Recommendations
                st.subheader("Installation Recommendations")
                st.markdown("""
                - **Panel Type**: Monocrystalline panels recommended for high efficiency.
                - **Mounting**: Flush mounts suitable for roof angle.
                - **Permits**: Check local regulations for net metering and safety standards.
                - **Maintenance**: Schedule annual cleaning and inverter checks.
                """)

                # Save analysis
                analysis_id = str(uuid.uuid4())
                analysis_data = {
                    "analysis_id": analysis_id,
                    "timestamp": datetime.now().isoformat(),
                    "location": location,
                    "analysis_result": analysis_result,
                    "roi_result": roi_result
                }
                with open(f"analysis_{analysis_id}.json", "w") as f:
                    json.dump(analysis_data, f, indent=2)

        except Exception as e:
            st.error(f"Error: {str(e)}")
            logger.error(f"Application error: {str(e)}")

if __name__ == "__main__":
    main()

2025-05-28 13:10:03.316 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-28 13:10:03.317 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-28 13:10:03.318 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-28 13:10:03.319 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-28 13:10:03.320 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-28 13:10:03.320 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-28 13:10:03.322 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-28 13:10:03.323 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar